## Handle Imports

In [1]:
import numpy as np
import pandas as pd
import heapq as pq
import csv
import statistics as stats
import copy
import math

## Load Up methods

### Load Countries

In [2]:
def load_countries():
    countries = {}

    with open("countries.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Country"]
            countries[key] = {name: float(value) for name, value in row.items() if name != "Country"}

    return countries

# LOAD Methods

In [3]:
def load_up():
    
    #load and create df
    countryList = load_countries()
    root_country_df = pd.DataFrame(countryList).transpose()
    
    return root_country_df

In [4]:
def get_country_names(df):
    return df.index.to_list()

### Jupyter Run

In [5]:
country_df = load_up()
root_country_df = load_up()
COUNTRY_NAMES = get_country_names(country_df)

In [6]:
COUNTRY_NAMES

['Atlantis', 'Dinotopoia', 'Erewhon', "King's Landing", 'The Vale']

## State Quality Function

# Set Maslow Constants

In [7]:
#different resource levels
L_ONE_RESOURCES = {'food': 1, 'water': 1}
L_TWO_RESOURCES = {'housing': 1, 'timber': 1, 'metallic alloy': 0.5, 'electronics': 2, 'potential fossil energy': 1} 
L_THREE_RESOURCES = {'community buildings': 0.05, 'jobs': 1, 'high school education': 1, 'college education': 1, 'universities': 1, 'marriages': 1} 
L_FOUR_RESOURCES = {'children': 2.5, 'renewable energy': 1}
L_FIVE_RESOURCES = {'food waste': -1, 'water waste': -1, 'land waste': -1, 'timber waste': -1, 'nobel prizes': 0.02}

In [8]:
#list of levels
LEV_LIST = [L_ONE_RESOURCES, L_TWO_RESOURCES, L_THREE_RESOURCES, L_FOUR_RESOURCES, L_FIVE_RESOURCES]

## Level Function

In [9]:
def leveldf(df, country, level, level_function_accessor):
    
    levelSat = False
    marUtility = False
    wasteRate = False
    mult = []
    average = 0

    for key, value in level_function_accessor[level-1].items():
        countryVal = df.loc[country, key]
        mult.append(countryVal/value)
        if countryVal < value:
            levelSat = True
            
        #marginal utility
        if countryVal > value*3:
            marUtility = True
            
            if countryVal > value * 6:
                wasteRate = True
                
    
    average = stats.mean(mult)
    
    if levelSat:
        average = average*0.01
        
    if marUtility:
        average = average*0.01
        
    if wasteRate:
        average = average*0.00001
        
    return average

### Level Function Unit Test

In [10]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,30.0,50.0,30.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,100.0,50.0,100.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


In [11]:
num = leveldf(country_df, 'Atlantis', 1, LEV_LIST)
num

4.000000000000001e-06

In [12]:
num = leveldf(country_df, 'Atlantis', 3, LEV_LIST)
num

6.810000000000002e-05

In [13]:
num = leveldf(country_df, 'Dinotopoia', 4, LEV_LIST)
num

1.0140000000000001e-05

In [14]:
num = leveldf(country_df, 'Atlantis', 5, LEV_LIST)
num

3.34e-08

## Maslow Function

In [15]:
def maslowLevelVals(df, country, level, level_function_accessor):
    
    maslowList = []
    
    ## normalize
    norm_df = df.copy()
    
    for row in range(len(norm_df)):
        
        values = norm_df.iloc[row]
        popVal = values[0]
        
        for vals in range(1, len(values)):
            values[vals] = values[vals]/popVal
    
    # maslow function
    for num in range(1, level+1):
        levValue = leveldf(norm_df, country, num, level_function_accessor)
        maslowList.append(levValue)
            
    return maslowList

In [16]:
maslowL = maslowLevelVals(country_df, 'Atlantis', 5, LEV_LIST)
maslowL

[2.0, 5.650000000000001e-09, 3.405e-08, 0.00026000000000000003, 1.67e-07]

In [17]:
def maslowList(lst):
    sumVal = 0
    sumVal = sumVal + lst[0] * 0.10
    sumVal = sumVal + lst[1] * 10
    sumVal = sumVal + lst[2] * 100
    sumVal = sumVal + lst[3] * 1000
    return sumVal

In [18]:
masVal = maslowList(maslowL)
masVal

0.46000346150000004

# Maslow

In [19]:
def maslowVal(df, country, level, LEV_LIST):
    
    lst = maslowLevelVals(df, country, level, LEV_LIST)
    val = maslowList(lst)
    return val

## Maslow Function Test Case

In [20]:
num = maslowVal(country_df, 'The Vale', 5, LEV_LIST)
num

10.406985373255814

In [21]:
num = maslowVal(country_df, 'Atlantis', 5, LEV_LIST)
num

0.46000346150000004

In [22]:
num = maslowVal(country_df, 'Dinotopoia', 5, LEV_LIST)
num

0.0038584066666666667

## Transfer Operation

### Transform Templates

In [23]:
HOUSING_TM = ['HOUSING', {'land': 1, 'population': 5, 'water': 5, 'metallic element': 1, 'timber': 5, 'metallic alloy': 3, 'potential fossil usable': 5}, {'housing': 1, 'housing waste': 1, 'timber waste': 1, 'population': 5, 'water': 4}]
ALLOYS_TM = ['ALLOYS', {'population': 1, 'metallic element': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'metallic alloy': 1, 'metallicAlloy waste': 1, 'water': 2}]
ELECTRONICS_TM = ['ELECTRONICS', {'population': 1, 'metallic element': 3, 'metallic alloy': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'electronics': 2, 'electronics waste': 1, 'water': 2}]
FARM_TM = ['FARM', {'population': 1, 'land' : 1, 'water': 3}, {'food': 5, 'population': 1}]
LOGGING_TM = ['LOGGING', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'timber': 5}]
PURIFY_WATER_TM = ['PURIFY_WATER', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'water': 5}]
FOSSIL_ENERGY_TM = ['FOSSIL_ENERGY', {'population': 5, 'potential fossil energy': 2}, {'population': 5, 'potential fossil usable': 1, 'potential fossil energy waste': 1}]
RENEWABLE_ENERGY_TM = ['RENEWABLE_ENERGY', {'population': 5, 'potential fossil usable': 3}, {'population': 5, 'renewable energy': 1, 'renewable energy waste': 1}]
COMMUNITY_BUILDING_TM = ['COMMUNITY_BUILDING', {'land': 1, 'population': 10, 'water': 5, 'metallic element': 3, 'timber': 8, 'metallic alloy': 5, 'potential fossil usable': 5}, {'community buildings': 1, 'housing waste': 1, 'timber waste': 1, 'metallicAlloy waste': 1, 'population': 10, 'water': 4}]
UNIVERSITY_TM = ['UNIVERSITY', {'land': 1, 'population': 50, 'water': 5, 'metallic element': 5, 'timber': 10, 'metallic alloy': 5, 'potential fossil usable': 5}, {'universities': 1, 'population': 50, 'water': 3, 'timber waste': 1, 'metallicAlloy waste': 1}]
JOB_HS_TM = ['JOB_HS', {'population': 25, 'high school education': 1}, {'population': 25, 'jobs': 1}]
JOB_C_TM = ['JOB_C', {'population': 50, 'college education': 1}, {'population': 50, 'jobs': 1}]
HIGHSCHOOL_ED_TM = ['HIGHSCHOOL_ED', {'population': 15, 'housing': 1, 'children': 1}, {'population': 16, 'housing': 1, 'high school education': 1}]
COLLEGE_ED_TM = ['COLLEGE_ED', {'population': 50, 'housing': 1, 'universities': 1, 'high school education': 1}, {'population': 50, 'housing': 1, 'universities': 1, 'college education': 1}]
MARRIAGE_TM = ['MARRIAGE', {'population': 2, 'housing': 1}, {'population': 2, 'housing': 1, 'marriages': 1}]
CHILDREN_TM = ['CHILDREN', {'marriages': 1, 'housing': 1}, {'marriages': 1, 'housing': 1, 'children': 2}]
NOBEL_PRIZE_TM = ['NOBEL_PRIZE', {'population': 1, 'universities': 10, 'college education': 50, 'potential fossil usable': 10}, {'population': 1, 'universities': 10, 'college education': 50, 'nobel prizes': 1}]

ALL_TEMPLATES_TRANSFORM = [HOUSING_TM, ALLOYS_TM, ELECTRONICS_TM, FARM_TM, LOGGING_TM, PURIFY_WATER_TM, FOSSIL_ENERGY_TM, 
                          RENEWABLE_ENERGY_TM, COMMUNITY_BUILDING_TM, UNIVERSITY_TM, JOB_HS_TM, JOB_C_TM, HIGHSCHOOL_ED_TM,
                          COLLEGE_ED_TM, MARRIAGE_TM, CHILDREN_TM, NOBEL_PRIZE_TM]

### Transform Function

In [24]:
def transform(df, country, transform_template):
    
    allowed = True
    
    #check if transform is possible
    for key in transform_template[1]:
        val = transform_template[1][key]
        if(df.loc[country, key] - val < 0):
            allowed = False
    
    if(allowed):
        #remove input resoures
        for key in transform_template[1]:
            val = transform_template[1][key]
            df.loc[country, key] -= val

        #add output resources
        for key in transform_template[2]:
            val = transform_template[2][key]
            df.loc[country, key] += val
    return allowed

### Transform Unit Test

In [25]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,30.0,50.0,30.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,100.0,50.0,100.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


In [26]:
transform(country_df, 'Atlantis', FARM_TM)

True

In [27]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,35.0,47.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,100.0,50.0,100.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


In [28]:
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)

True

In [29]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,35.0,47.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,130.0,32.0,94.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


In [30]:
transform(country_df, 'The Vale', FARM_TM)

True

In [31]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,35.0,47.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,135.0,29.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


# Transfer Function Definition

### Transfer Templates

In [32]:
FOOD_TR = ['FOOD', {'food': 5}]
WATER_TR = ['WATER', {'water': 5}]
TIMBER_TR = ['TIMBER', {'timber': 5}]
METALLIC_ELEMENT_TR = ['METALLIC ELEMENT', {'metallic element':5}]
METALLIC_ALLOY_TR = ['METALLIC ALLOY', {'metallic alloy':5}]
ELECTRONICS_TR = ['ELECTRONICS', {'electronics':5}]
POTENTIAL_FOSSIL_ENERGY_TR = ['POTENTIAL_FOSSIL_ENERGY_TR',{'potential fossil energy':5}]
POTENTIAL_FOSSIL_USABLE_TR = ['POTENTIAL_FOSSIL_USABLE_TR',{'potential fossil usable':5}]
RENEWABLE_ENERGY_TR = ['RENEWABLE_ENERGY_TR',{'renewable energy':5}]

In [33]:
ALL_TEMPLATES_TRANSFER = [FOOD_TR,WATER_TR,TIMBER_TR,METALLIC_ELEMENT_TR,METALLIC_ALLOY_TR,ELECTRONICS_TR,
                          POTENTIAL_FOSSIL_ENERGY_TR,POTENTIAL_FOSSIL_USABLE_TR,RENEWABLE_ENERGY_TR]

### Transfer Function

In [34]:
def transfer(df, country1, country2, transfer_template):

    allowed = True

    #check for validity
    
    if country1==country2:
        allowed = False

    for key in transfer_template[1]:
        val = transfer_template[1][key]
        if (df.loc[country1,key] - val < 0):
            allowed = False
            
    if(allowed):
        #remove resource from country 1
        for key in transfer_template[1]:
            val = transfer_template[1][key]
            df.loc[country1, key] -= val

        #add resource to country 2
        for key in transfer_template[1]:
            val = transfer_template[1][key]
            df.loc[country2, key] += val
            
    return allowed

## Transfer Function Test Case

In [35]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,35.0,47.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,135.0,29.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


In [36]:
transfer(country_df, 'Atlantis', 'The Vale', WATER_TR)

True

In [37]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,35.0,42.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,135.0,34.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


In [38]:
transfer(country_df, 'The Vale', 'Atlantis', WATER_TR)

True

In [39]:
transfer(country_df, 'The Vale', 'Atlantis', WATER_TR)

True

In [40]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,35.0,52.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,135.0,24.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


# Trade Function Definition

In [41]:
def trade(df, country1, country2, transfer1, transfer2):
    # A trade deal between two countries country1 and country2
    # for which country uses transfer1 and country2 uses transfer2
    if transfer1 == transfer2:
        return False
    if transfer(df, country1, country2, transfer1):
        if not transfer(df, country2, country1, transfer2):
            transfer(df, country2, country1, transfer1)
            return False
        else:
            return True
    else:
        return False

## Trade Function Test Case

In [42]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,35.0,52.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,135.0,24.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


In [43]:
trade(country_df,'Atlantis','The Vale',FOOD_TR, WATER_TR)

True

In [44]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,30.0,57.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,140.0,19.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


In [45]:
trade(country_df,'Atlantis','The Vale', WATER_TR, FOOD_TR)

True

In [46]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,35.0,52.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,135.0,24.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


# Node Definition

In [47]:
class Node:
    def __init__(self, state_quality, exp_util, sched, countrydf):
        self.state_quality = state_quality
        self.exp_util = exp_util
        self.sched = sched
        self.countrydf = countrydf
    def getSqual(self):
        return self.state_quality
    def getExutil(self):
        return self.exp_util
    def getSched(self):
        return self.sched
    def getCountrydf(self):
        return self.countrydf
    def __lt__(self, other):
        return self.exp_util < other.exp_util
    def __str__(self):
        s = "Node with State Quality: {}\nExpected Utility: {}\nSchedule: {}\nDataframe:\n{}".format(self.state_quality, self.exp_util, self.sched, self.countrydf)
        return s

# Bounded Priority Queue Definition

In [48]:
class BoundedPriorityQueue:
    # can change bound
    def __init__(self, queue=list(), bound=2):
        self.bound = bound
        self.queue = queue
    def push(self, priority, item):
        if len(self.queue) == self.bound:
            pq.heappushpop(self.queue, (priority, item))
        else:
            pq.heappush(self.queue, (priority, item))
    def pop(self):
        if len(self.queue) >= 1:
            return pq.heappop(self.queue)[-1]
        print("no nodes on fringe")
    ##maybe delete    
    def items(self):
        return list(item for _,_, item in self.queue)
    def clear(self):
        self.queue.clear()
    def len(self):
        return len(self.queue)
    def getState(self, index):
        if len(self.queue) > index:
            return self.queue[index][1]
        print("index out of bound")
    def pState(self, index):
        if len(self.queue) > index:
            print(self.queue[index][1])
        print("index out of bound")
    def getPriority(self, index):
        if len(self.queue) > index:
            return self.queue[index][0]
        print("index out of bound")
    def popBest(self):
        if len(self.queue) >= 1:
            return pq.nlargest(1, self.queue)[0][1]
        print("no nodes on fringe")
    def copy(self):
        return BoundedPriorityQueue(self.queue.copy())

## Generate Succcessors

In [49]:
def generate_successors(parentNode, my_country, LEV_LIST):
    
    successors = BoundedPriorityQueue()
    prevSched = parentNode.getSched()
    
    for resource in ALL_TEMPLATES_TRANSFORM:
        
        new_df = parentNode.getCountrydf().copy()
        
        if transform(new_df, my_country, resource):
            state_qual = maslowVal(new_df, my_country, 5, LEV_LIST)
            new_schedule = prevSched.copy()
            new_state = Node(state_qual, 0, new_schedule, new_df)
            new_state.sched.append("TRANSFORM: " + resource[0])
            new_state.exp_util = expected_utility_transform(new_state, my_country, LEV_LIST)
            successors.push(new_state.exp_util, new_state)


    for resource_1 in ALL_TEMPLATES_TRANSFER:
        for resource_2 in ALL_TEMPLATES_TRANSFER:
            for other_country in COUNTRY_NAMES:
                
                new_df = parentNode.getCountrydf().copy()
                
                if trade(new_df, my_country, other_country, resource_1, resource_2):
                    state_qual = maslowVal(new_df, my_country, 5, LEV_LIST)
                    state_qual_other = maslowVal(new_df, other_country, 5, LEV_LIST)
                    new_schedule = prevSched.copy()
                    new_state = Node(state_qual, 0, new_schedule, new_df)
                    trade_name = "TRADE: " + my_country + " GIVES " + resource_1[0] + ", " + other_country + " GIVES " + resource_2[0]
                    new_state.sched.append(trade_name)
                    new_state.exp_util = expected_utility_trade(L, k, x_0, new_state, my_country, LEV_LIST, other_country)
                    successors.push(new_state.exp_util, new_state)
                    
    return successors

# Reward Functions

In [50]:
def undiscounted_reward(node, my_country,LEV_LIST):
    root_node_score = maslowVal(root_country_df, my_country, 5, LEV_LIST)
    cur_node_score = maslowVal(node.getCountrydf(), my_country, 5, LEV_LIST)
    return cur_node_score - root_node_score

In [51]:
GAMMA = 0.05

def discounted_reward(node, my_country, LEV_LIST):
    return (GAMMA ** len(node.getSched()))*undiscounted_reward(node, my_country, LEV_LIST)

# Probability function

In [52]:
L = 1.0
k = 1.0
x_0 = 0.0

def probability(L, k, node, other_country, LEV_LIST, x_0):
    return (L/(1+math.exp(-k*(discounted_reward(node, other_country, LEV_LIST)-x_0))))

In [53]:
COST_FAILURE = -0.05

def expected_utility_trade(L, k, x_0, node, my_country, LEV_LIST, other_country):
    prob = probability(L, k, node, other_country, LEV_LIST, x_0)
    success = prob * discounted_reward(node, other_country, LEV_LIST)
    fail = (1 - prob) * COST_FAILURE
    other_country_utility = success + fail
    prob = probability(L, k, node, my_country, LEV_LIST, x_0)
    success = prob * discounted_reward(node, my_country, LEV_LIST)
    fail = (1 - prob) * COST_FAILURE
    my_country_utility = success + fail
    total = (other_country_utility+my_country_utility)/2
    return total

In [54]:
PROB = 0.95
COST_FAILURE = -0.05
def expected_utility_transform(node, my_country, LEV_LIST):
    success = PROB * discounted_reward(node, my_country, LEV_LIST)
    fail = (1-PROB)*COST_FAILURE 
    return success + fail

# Node Test Cases 

In [55]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,renewable energy,food waste,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste
Atlantis,20.0,35.0,52.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,100.0,-5.0,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,200.0,-2.0,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,100.0,-10.0,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,150.0,-3.0,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0
The Vale,43.0,135.0,24.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,80.0,-10.0,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0


In [56]:
state_qual = 1.2
exp_util = 3.0
sched = ['ROOT STATE']
test_node = Node(state_qual, exp_util, sched, country_df)

In [57]:
expected_utility_transform(test_node, 'Atlantis', LEV_LIST)

-0.001668750000000004

In [58]:
successors = generate_successors(test_node, 'Atlantis', LEV_LIST)

In [59]:
print(successors.len())
suc1 = successors.pop()

2


In [60]:
yes = BoundedPriorityQueue()
yes.push(1, test_node)
yes.push(1, test_node)
print(yes.len())
print(yes.pop())

2
Node with State Quality: 1.2
Expected Utility: 3.0
Schedule: ['ROOT STATE']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale                   150.0           100.0        100.0   

                potential fossil energy  ...  renewable energy  food waste  \
Atlantis                         

In [61]:
print(suc1.getSched())

['ROOT STATE', 'TRANSFORM: FARM']


In [62]:
successors.getPriority(1)

index out of bound


## Search Function

In [63]:
def search(root_node, my_country,level_accessor, max_iter):
    
    num = 1
    
    fringe = generate_successors(root_node, my_country, level_accessor)
    newfringe = BoundedPriorityQueue()
    for i in range(1, max_iter):
        fringe_bound = fringe.len()
        for j in range(0,fringe_bound):
            successors = generate_successors(fringe.pop(), my_country, level_accessor)
            for k in range(0,successors.len()-1):
                newfringe.push(successors.getState(k).getExutil(),successors.getState(k))
                print("Explored Node: " + str(num))
                num = num + 1
        print("Level: " + str(i))
        print(newfringe.popBest())
        fringe = newfringe.copy()
        newfringe.clear()
    return fringe.popBest()

### Search Function Test Cases

In [64]:
nodeBest = search(test_node, 'Atlantis', LEV_LIST, 10)

Explored Node: 1
Explored Node: 2
Level: 1
Node with State Quality: 1.2
Expected Utility: 3.0
Schedule: ['ROOT STATE']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale                   150.0           100.0        100.0   

                potential fossil energy  ...  renewable energy  food w

Explored Node: 9
Explored Node: 10
Level: 5
Node with State Quality: 0.5125034615
Expected Utility: -0.0024999844140625024
Schedule: ['ROOT STATE', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   45.0   56.0   27.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale       

Explored Node: 15
Explored Node: 16
Level: 8
Node with State Quality: 0.5350034615000001
Expected Utility: -0.0024999999972167993
Schedule: ['ROOT STATE', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   55.0   55.0   25.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's L

In [65]:
print(nodeBest)

Node with State Quality: 0.5475034615000001
Expected Utility: -0.0024999999998376486
Schedule: ['ROOT STATE', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   55.0   60.0   25.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing            

In [66]:
nodeBest = search(test_node, 'The Vale', LEV_LIST, 10)

Explored Node: 1
Explored Node: 2
Level: 1
Node with State Quality: 10.44188758255814
Expected Utility: -0.002495855362645351
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale                   150.0           100.0  

Explored Node: 7
Explored Node: 8
Level: 4
Node with State Quality: 10.790724791860464
Expected Utility: -0.0024999943038680073
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing      

Explored Node: 13
Explored Node: 14
Level: 7
Node with State Quality: 11.139562001162789
Expected Utility: -0.0024999999986407293
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0       

In [67]:
print(nodeBest)

Node with State Quality: 11.372120140697675
Expected Utility: -0.0024999999999955253
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0      

In [68]:
nodeBest = search(test_node, 'Erewhon', LEV_LIST, 10)

Explored Node: 1
Explored Node: 2
Level: 1
Node with State Quality: 0.49050010317460324
Expected Utility: -0.002488690476190478
Schedule: ['ROOT STATE', 'TRANSFORM: FARM']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   45.0   57.0   19.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale                   150.0           100.0        100.0   

                p

Explored Node: 9
Explored Node: 10
Level: 5
Node with State Quality: 0.5238334365079366
Expected Utility: -0.0024999994345238116
Schedule: ['ROOT STATE', 'TRANSFORM: FARM', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   55.0   61.0   17.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0       

Explored Node: 15
Explored Node: 16
Level: 8
Node with State Quality: 0.5428810555555557
Expected Utility: -0.0024999999998939755
Schedule: ['ROOT STATE', 'TRANSFORM: FARM', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   60.0   63.0   16.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0          

In [69]:
print(nodeBest)

Node with State Quality: 0.545262007936508
Expected Utility: -0.00249999999999448
Schedule: ['ROOT STATE', 'TRANSFORM: FARM', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   60.0   63.0   16.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75

In [70]:
nodeBest = search(test_node, 'Dinotopoia', LEV_LIST, 10)

Explored Node: 1
Explored Node: 2
Level: 1
Node with State Quality: 0.004025073333333334
Expected Utility: -0.002499604166666669
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY_WATER']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   85.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale                   150.0           100.0        100.0   

        

Explored Node: 9
Explored Node: 10
Level: 5
Node with State Quality: 0.00449174
Expected Utility: -0.0024999999905989606
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER']
Dataframe:
                population   food  water  land  timber  housing  \
Atlantis              20.0   35.0   52.0  29.0    20.0     40.0   
Dinotopoia            15.0   45.0   89.0  98.0    40.0    230.0   
Erewhon               21.0   40.0   60.0  20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0  50.0    40.0     80.0   
The Vale              43.0  135.0   24.0  93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0

Explored Node: 15
Explored Node: 16
Level: 8
Node with State Quality: 0.004791740000000001
Expected Utility: -0.0024999999999982705
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER']
Dataframe:
                population   food  water  land  timber  housing  \
Atlantis              20.0   35.0   52.0  29.0    20.0     40.0   
Dinotopoia            15.0   55.0   88.0  96.0    40.0    230.0   
Erewhon               21.0   40.0   60.0  20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0  50.0    40.0     80.0   
The Vale              43.0  135.0   24.0  93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0  

In [71]:
print(nodeBest)

Node with State Quality: 0.004858406666666668
Expected Utility: -0.0024999999999999094
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY_WATER', 'TRANSFORM: FARM']
Dataframe:
                population   food  water  land  timber  housing  \
Atlantis              20.0   35.0   52.0  29.0    20.0     40.0   
Dinotopoia            15.0   60.0   85.0  95.0    40.0    230.0   
Erewhon               21.0   40.0   60.0  20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0  50.0    40.0     80.0   
The Vale              43.0  135.0   24.0  93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Land

In [72]:
nodeBest = search(test_node, 'The Vale', LEV_LIST, 10)

Explored Node: 1
Explored Node: 2
Level: 1
Node with State Quality: 10.44188758255814
Expected Utility: -0.002495855362645351
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale                   150.0           100.0  

Explored Node: 7
Explored Node: 8
Level: 4
Node with State Quality: 10.790724791860464
Expected Utility: -0.0024999943038680073
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing      

Explored Node: 13
Explored Node: 14
Level: 7
Node with State Quality: 11.139562001162789
Expected Utility: -0.0024999999986407293
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0       

In [73]:
print(nodeBest)

Node with State Quality: 11.372120140697675
Expected Utility: -0.0024999999999955253
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0      

In [74]:
nodeBest = search(test_node, 'The Vale', LEV_LIST, 100)

Explored Node: 1
Explored Node: 2
Level: 1
Node with State Quality: 10.44188758255814
Expected Utility: -0.002495855362645351
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale                   150.0           100.0  

Explored Node: 7
Explored Node: 8
Level: 4
Node with State Quality: 10.790724791860464
Expected Utility: -0.0024999943038680073
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing      

Explored Node: 13
Explored Node: 14
Level: 7
Node with State Quality: 11.139562001162789
Expected Utility: -0.0024999999986407293
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0       

Explored Node: 19
Explored Node: 20
Level: 10
Node with State Quality: 11.488399210465117
Expected Utility: -0.0024999999999997516
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                  

Explored Node: 25
Explored Node: 26
Level: 13
Node with State Quality: 11.558166326744187
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              20.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            15.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               21.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        80.0  200.0  100.0   50.0    40.0     80.0   
The Vale              43.0  135.0   22.0   91.0   100.0     72.0   

                metallic element

...

Explored Node: 181
Explored Node: 182
Level: 91
Node with State Quality: 11.55816453139535
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: ALLOYS', 'TRANSFORM: ALLOYS', 'TRANSFORM: L

Explored Node: 185
Explored Node: 186
Level: 93
Node with State Quality: 11.558164577906977
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: ALLOYS', 'TRANSFORM: ALLOYS', 'TRANSFORM: 

Explored Node: 189
Explored Node: 190
Level: 95
Node with State Quality: 11.558164624418605
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: ALLOYS', 'TRANSFORM: ALLOYS', 'TRANSFORM: 

Explored Node: 193
Explored Node: 194
Level: 97
Node with State Quality: 11.558164670930234
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: ALLOYS', 'TRANSFORM: ALLOYS', 'TRANSFORM: 

Explored Node: 197
Explored Node: 198
Level: 99
Node with State Quality: 11.558164717441862
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: ALLOYS', 'TRANSFORM: ALLOYS', 'TRANSFORM: 

In [75]:
print(nodeBest)

Node with State Quality: 11.558164717441862
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: RENEWABLE_ENERGY', 'TRANSFORM: CHILDREN', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: HOUSING', 'TRANSFORM: ALLOYS', 'TRANSFORM: ALLOYS', 'TRANSFORM: LOGGING', 'TRANSFORM: LOGGING', 'TRANSFORM: LOGG

# Driver

In [ ]:
def driver(country, tree_size):
    #load in countries
    country_df = load_up()
    COUNTRY_NAMES = get_country_names(country_df)
    
    #different resource levels
    L_ONE_RESOURCES = {'food': 1, 'water': 1}
    L_TWO_RESOURCES = {'housing': 1, 'timber': 1, 'metallic alloy': 0.5, 'electronics': 3, 'potential fossil energy': 1} 
    L_THREE_RESOURCES = {'community buildings': 0.05, 'jobs': 1, 'high school education': 1, 'college education': 1, 'universities': 1, 'marriages': 1} 
    L_FOUR_RESOURCES = {'children': 2.5, 'renewable energy': 1}
    L_FIVE_RESOURCES = {'food waste': -1, 'water waste': -1, 'land waste': -1, 'timber waste': -1, 'nobel prizes': 0.02}
    
    LEV_LIST = [LEVELONERESOURCES, L_TWO_RESOURCES, L_THREE_RESOURCES, L_FOUR_RESOURCES, L_FIVE_RESOURCES]
    
    #transfer templates
    HOUSING_TM = ['HOUSING', {'land': 1, 'population': 5, 'water': 5, 'metallic element': 1, 'timber': 5, 'metallic alloy': 3, 'potential fossil usable': 5}, {'housing': 1, 'housing waste': 1, 'timber waste': 1, 'population': 5, 'water': 4}]
    ALLOYS_TM = ['ALLOYS', {'population': 1, 'metallic element': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'metallic alloy': 1, 'metallicAlloy waste': 1, 'water': 2}]
    ELECTRONICS_TM = ['ELECTRONICS', {'population': 1, 'metallic element': 3, 'metallic alloy': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'electronics': 2, 'electronics waste': 1, 'water': 2}]
    FARM_TM = ['FARM', {'population': 1, 'land' : 1, 'water': 3}, {'food': 5, 'population': 1}]
    LOGGING_TM = ['LOGGING', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'timber': 5}]
    PURIFY_WATER_TM = ['PURIFY_WATER', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'water': 5}]
    FOSSIL_ENERGY_TM = ['FOSSIL_ENERGY', {'population': 5, 'potential fossil energy': 2}, {'population': 5, 'potential fossil usable': 1, 'potential fossil energy waste': 1}]
    RENEWABLE_ENERGY_TM = ['RENEWABLE_ENERGY', {'population': 5, 'potential fossil usable': 3}, {'population': 5, 'renewable energy': 1, 'renewable energy waste': 1}]
    COMMUNITY_BUILDING_TM = ['COMMUNITY_BUILDING', {'land': 1, 'population': 10, 'water': 5, 'metallic element': 3, 'timber': 8, 'metallic alloy': 5, 'potential fossil usable': 5}, {'community buildings': 1, 'housing waste': 1, 'timber waste': 1, 'metallicAlloy waste': 1, 'population': 10, 'water': 4}]
    UNIVERSITY_TM = ['UNIVERSITY', {'land': 1, 'population': 50, 'water': 5, 'metallic element': 5, 'timber': 10, 'metallic alloy': 5, 'potential fossil usable': 5}, {'universities': 1, 'population': 50, 'water': 3, 'timber waste': 1, 'metallicAlloy waste': 1}]
    JOB_HS_TM = ['JOB_HS', {'population': 25, 'high school education': 1}, {'population': 25, 'jobs': 1}]
    JOB_C_TM = ['JOB_C', {'population': 50, 'college education': 1}, {'population': 50, 'jobs': 1}]
    HIGHSCHOOL_ED_TM = ['HIGHSCHOOL_ED', {'population': 15, 'housing': 1, 'children': 1}, {'population': 16, 'housing': 1, 'high school education': 1}]
    COLLEGE_ED_TM = ['COLLEGE_ED', {'population': 50, 'housing': 1, 'universities': 1, 'high school education': 1}, {'population': 50, 'housing': 1, 'universities': 1, 'college education': 1}]
    MARRIAGE_TM = ['MARRIAGE', {'population': 2, 'housing': 1}, {'population': 2, 'housing': 1, 'marriages': 1}]
    CHILDREN_TM = ['CHILDREN', {'marriages': 1, 'housing': 1}, {'marriages': 1, 'housing': 1, 'children': 2}]
    NOBEL_PRIZE_TM = ['NOBEL_PRIZE', {'population': 1, 'universities': 10, 'college education': 50, 'potential fossil usable': 10}, {'population': 1, 'universities': 10, 'college education': 50, 'nobel prizes': 1}]

    #total transform template list
    ALL_TEMPLATES_TRANSFORM = [HOUSING_TM, ALLOYS_TM, ELECTRONICS_TM, FARM_TM, LOGGING_TM, PURIFY_WATER_TM, FOSSIL_ENERGY_TM, 
                          RENEWABLE_ENERGY_TM, COMMUNITY_BUILDING_TM, UNIVERSITY_TM, JOB_HS_TM, JOB_C_TM, HIGHSCHOOL_ED_TM,
                          COLLEGE_ED_TM, MARRIAGE_TM, CHILDREN_TM, NOBEL_PRIZE_TM]
    
    #transfer templates
    FOOD_TR = ['FOOD', {'food': 5}]
    WATER_TR = ['WATER', {'water': 5}]
    TIMBER_TR = ['TIMBER', {'timber': 5}]
    METALLIC_ELEMENT_TR = ['METALLIC ELEMENT', {'metallic element':5}]
    METALLIC_ALLOY_TR = ['METALLIC ALLOY', {'metallic alloy':5}]
    ELECTRONICS_TR = ['ELECTRONICS', {'electronics':5}]
    POTENTIAL_FOSSIL_ENERGY_TR = ['POTENTIAL_FOSSIL_ENERGY_TR',{'potential fossil energy':5}]
    POTENTIAL_FOSSIL_USABLE_TR = ['POTENTIAL_FOSSIL_USABLE_TR',{'potential fossil usable':5}]
    RENEWABLE_ENERGY_TR = ['RENEWABLE_ENERGY_TR',{'renewable energy':5}]
    
    #total transfer template list
    ALL_TEMPLATES_TRANSFER = [FOOD_TR,WATER_TR,TIMBER_TR,METALLIC_ELEMENT_TR,METALLIC_ALLOY_TR,ELECTRONICS_TR,
                          POTENTIAL_FOSSIL_ENERGY_TR,POTENTIAL_FOSSIL_USABLE_TR,RENEWABLE_ENERGY_TR]
    
    #define class
    class Node:
        def __init__(self, state_quality, exp_util, sched, countrydf):
            self.state_quality = state_quality
            self.exp_util = exp_util
            self.sched = sched
            self.countrydf = countrydf
        def getSqual(self):
            return self.state_quality
        def getExutil(self):
            return self.exp_util
        def getSched(self):
            return self.sched
        def getCountrydf(self):
            return self.countrydf
        def __lt__(self, other):
            return self.exp_util < other.exp_util
        def __str__(self):
            s = "Node with State Quality: {}\nExpected Utility: {}\nSchedule: {}\nDataframe:\n{}".format(self.state_quality, self.exp_util, self.sched, self.countrydf)
            return s

        
    #define priority queue
    class BoundedPriorityQueue:
        def __init__(self, bound=None):
            self.bound = bound
            self.queue = list()
        def push(self, priority, item):
            if len(self.queue) == self.bound:
                pq.heappushpop(self.queue, (priority, item))
            else:
                pq.heappush(self.queue, (priority, item))
        def pop(self):
            if len(self.queue) >= 1:
                return pq.heappop(self.queue)[-1]
            print("no nodes on fringe")
        def clear(self):
            self.queue.clear()
        def len(self):
            return len(self.queue)
        def getState(self, index):
            if len(self.queue) > index:
                return self.queue[index][1]
            print("index out of bound")
        def pState(self, index):
            if len(self.queue) > index:
                print(self.queue[index][1])
            print("index out of bound")
        def getPriority(self, index):
            if len(self.queue) > index:
                return self.queue[index][0]
            print("index out of bound")
        def popBest(self):
            if len(self.queue) >= 1:
                return pq.nlargest(1, self.queue)[0][1]
            print("no nodes on fringe")
    
    #define root node
    root_node = Node(0, 0, [], country_df)
    successors = generate_successors(root_node, country)
    
    while tree_size > 1:
        for node in successors:
            #search functiom
            tree_size = tree_size - 1